In [1]:
import sys
sys.path.append("../../")
from elman import Elman
from rnn_disf_detection.load.load import load_data_from_array, get_tag_data_from_corpus_file, load_word_rep
from rnn_disf_detection.utils.tools import contextwinbackwards

In [2]:
nclasses = 43
vocsize = 9068
possize = 125 
s = { 'emb_dimension' : 50, 'nhidden' : 50, 'acoustic' : 350, 'window' : 3, 'update_embeddings' : True }

In [3]:
rnn = Elman(ne = vocsize,
                        de = s['emb_dimension'],
                        nh = s['nhidden'],
                        na = 0,
                        nc = nclasses,
                        cs = s['window'],
                        npos = possize,
                        update_embeddings=s['update_embeddings'])

/Library/Python/2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [4]:
print (possize * 3) + (s['emb_dimension']* 3)

525


In [5]:
test_file = "../data/audio/switchboard/4646A.npy"

In [6]:
import numpy as np

In [7]:
data = np.load(test_file)

In [8]:
_, acoustic_data, lex_data, pos_data, indices, labels = load_data_from_array(data, s['acoustic'])

In [9]:
from theano import tensor as T
import theano

y = T.concatenate((rnn.emb[rnn.idxs].reshape((rnn.idxs.shape[0], s['emb_dimension']*3)),\
                               rnn.pos[rnn.pos_idxs].reshape((rnn.pos_idxs.shape[0],possize*3))), 1)
def print_y():
    return y
    
printy = theano.function( inputs  = [rnn.idxs, rnn.pos_idxs],
                                       outputs = y )    

In [16]:
word_rep = load_word_rep("../data/tag_representations/swbd_word_rep.csv", dimension=None, word_rep_type="one_hot")
#print word_rep
idx2word  = dict((k,v) for v,k in word_rep.iteritems())    
    
acoustic = True
lr = 0.005
current_index = 0

_, acoustic_data, lex_data, pos_data, indices, labels = load_data_from_array(data, s['acoustic'])
nw = acoustic_data.shape[1] # number of examples in this dialogue
loss= 0
if acoustic:

    test = 0
    #load in the data to shared vars, can use 'set value too'
    #self.lexical_data = self.shared_dataset(lex_data) #loads dummy data set as a shared variable
    #self.pos_data = self.shared_dataset(pos_data)
    #self.acoustic_data = data = self.shared_dataset(acoustic_data,dtype=theano.config.floatX)
    #self.labels = self.shared_dataset(labels)
    for start,stop in indices:
        current_index+=1
        test+=1 #TODO for testing
        if test > 50: break #TODO for testing
        print "llllll"
        for row in lex_data[start:stop+1]:
            print "r"
            for c in row:
                print c
                print idx2word.get(c)
        print 'words'
        print lex_data[start:stop+1,:].shape
        print lex_data[start:stop+1].shape
        newwords = np.swapaxes(lex_data[start:stop+1,:],0,1)
        print newwords.shape
        print newwords
        print 'pos'
        print pos_data[start:stop+1,:].shape
        print pos_data[start:stop+1].shape
        newpos = np.swapaxes(pos_data[start:stop+1,:],0,1)
        print newpos.shape
        print newpos
        #print 'acoust'
        #print acoustic_data[:,start:stop+1].shape
        #newacoust =  np.swapaxes(acoustic_data[:,start:stop+1],0,1)
        #print newacoust.shape
        print 'label', labels[stop]
        
        print printy(lex_data[start:stop+1],pos_data[start:stop+1])[:,525-possize:]
        
        raw_input()
        #print np.asarray(acoustic_data[:,start:stop+1],dtype='float32')
        
        
        #x = rnn.train(lex_data[start:stop+1,:],pos_data[start:stop+1,:],labels[stop],lr)
        #loss+=x
        ##rnn.normalize()

        #print '[learning] >> %2.2f%%'%((stop+1)*100./nw),'of file {} / {}'.format(i+1,len(dialogues)),\
        #'completed in %.2f (sec) <<\r'%(time.time()-tic),
        sys.stdout.flush()
print "current train_loss (may include reg)is", loss/float(current_index)


llllll
r
-1
None
-1
None
2548
do
words
(1, 3)
(1, 3)
(3, 1)
[[  -1]
 [  -1]
 [2548]]
pos
(1, 3)
(1, 3)
(3, 1)
[[-1]
 [-1]
 [65]]
label 6.0
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]

llllll
r
-1
None
-1
None
2548
do
r
-1
None
2548
do
3391
you
words
(2, 3)
(2, 3)
(3, 2)
[[  -1   -1]
 [  -1 2548]
 [2548 3391]]
pos
(2, 3)
(2, 3)
(3, 2)
[[-1 -1]
 [-1 65]
 [65 16]]
label 4.0
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0. 

In [ ]:
test_file2 = "../data/audio/switchboard/4619B.npy"

In [14]:
dialogues = [("",test_file), ("",test_file2)]

In [15]:
data = np.load(test_file2)

In [16]:
data[:][350+2]

array([  57.,   57.,  124.,   57.,    0.,   58.,  119.,   57.,  100.,
         68.,  119.,  100.,   68.,   57.,   57.,   16.,   57.,    0.,
         58.,    0.,   58.,   17.,   82.,   78.,   16.,   67.,  119.,
         16.,   78.,   71.,  119.,    4.,   68.,   57.,   78.,   57.,
        100.,   19.,  119.,    0.,   65.,   57.,   64.,  119.,  100.,
         16.,   65.,   80.,  113.,  119.,  100.,   68.,   22.,   16.,
         63.,  100.,   68.,  119.,   68.,   78.,   57.,   22.,   16.,
         63.,  100.,   19.,  107.,  113.,  107.,  113.,   16.,   57.,
         17.,    4.,   78.,   22.,   49.,   63.,   57.,    4.,   78.,
          4.,   22.,   49.,   63.,    4.,   78.,    4.,   19.,   57.,
         62.,   16.,   65.,   16.,  113.,  100.,   68.,  119.,  100.,
         16.,   57.,   57.,  119.,  100.,   68.,  119.,   68.,   16.,
         65.,   65.,   16.,  113.,  119.,  119.,  100.,   68.,  119.,
        100.,   82.,    3.,   96.,   17.,   64.,  100.,   68.,  119.,
        119.,  100.,

In [17]:
rnn.fit(dialogues, 0.005, acoustic=True, load_data=True)

KeyboardInterrupt: 

In [12]:
test = 0
for start,stop in indices:
    test+=1 #TODO for testing
    if test > 50: break #TODO for testing
    #print type(lex_data[start:stop+1]), lex_data[start:stop+1].shape
    #print type(pos_data[start:stop+1]), pos_data[start:stop+1].shape
    #print type(acoustic_data[:,start:stop+1]), acoustic_data[:,start:stop+1].shape
    #print type(labels[stop])
    x = rnn.train(lex_data[start:stop+1],pos_data[start:stop+1],np.asarray(acoustic_data[:,start:stop+1],dtype='float32'),labels[stop],0.005)
    #loss+=x
    rnn.normalize()

In [13]:
#np.swapaxes(acoustic_data[:,start:stop+1], 0, 1).shape
print x

2.20022251803


In [137]:
print fr.shape, ac.shape, le.shape, po.shape, ind.shape, labels.shape

(1047,) (350, 1047) (1047, 3) (1047, 3) (1047, 2) (354,)


In [146]:
start, stop = ind[(100)]

In [148]:
le[start:stop]

array([[7613, 4904, 6999],
       [4904, 6999,  707],
       [6999,  707,  355],
       [ 707,  355, 3064],
       [ 355, 3064, 4258],
       [3064, 4258, 1904],
       [4258, 1904, 2980],
       [1904, 2980,  840]])

In [30]:
groundtruth_filename = "../data/disfluency_detection/switchboard/swbd_heldout_partial_data.csv"

id,ind, gold_words,_,gold_labels = get_tag_data_from_corpus_file(groundtruth_filename, representation="1_trp")

loading data ../data/disfluency_detection/switchboard/swbd_heldout_partial_data.csv
loaded 5732 sequences


In [31]:
turn_end = True
limit = 100
for word, label in zip(sum(gold_words,()),sum(gold_labels,())):
    if turn_end == True:
        assert("<t" in label)
    if turn_end == False:
        assert("<c" in label)
    if "c>" in label:
        turn_end = False
    else:
        turn_end = True
    print word, label
    limit-=1
    if limit <0 : break

do <f/><tc>
you <f/><cc>
know <f/><cc>
anyone <f/><cc>
that <f/><cc>
uh <e/><cc>
is <rms id="7"/><cc>
is <rps id="7"/><rpnrep id="7"/><cc>
in <f/><cc>
a <f/><cc>
nursing <f/><cc>
home <f/><cc>
or <f/><cc>
has <f/><cc>
ever <f/><cc>
been <f/><cc>
in <f/><cc>
$unc$one <f/><ct>
no <f/><tt>
but <f/><tc>
i <rms id="2"/><cc>
my <rps id="2"/><cc>
grandparents <rpnsub id="2"/><cc>
were <f/><cc>
looking <f/><cc>
into <f/><cc>
it <f/><cc>
before <f/><ct>
so <f/><tc>
i <f/><cc>
know <f/><cc>
what <f/><cc>
theyve <f/><cc>
said <f/><ct>
uh-huh <f/><tt>
uh-huh <f/><tt>
well <e/><tc>
im <f/><cc>
trying <f/><cc>
to <f/><cc>
think <f/><ct>
my <f/><tc>
uh <e/><cc>
uh <e/><cc>
wifes <f/><cc>
grandmother <f/><cc>
had <f/><cc>
alzheimers <f/><ct>
and <f/><tc>
they <f/><cc>
were <f/><cc>
going <f/><cc>
to <f/><cc>
put <f/><cc>
her <f/><cc>
into <f/><cc>
a <rms id="9"/><cc>
a <rps id="9"/><rpnrep id="9"/><cc>
nursing <f/><cc>
home <f/><ct>
and <f/><tc>
uh <e/><cc>
they <rms id="3"/><cc>
when <rps id="3"/><cc